In [1]:
import sys
sys.path.append("..")

In [2]:

from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import numpy as np
from satreeverify.rf_utils import *
from satreeverify.utils import disc_data, create_var_x, get_x_adv

/home/masoud/anaconda3/lib/python3.8/site-packages/xgboost/compat.py:31: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


# Train a model with Random Forest

In [3]:
data = load_breast_cancer()
feature_names = data["feature_names"]

In [4]:
X = data['data']
y = data["target"] == 1
feature_names = data["feature_names"]
ntrees = 50
clf = RandomForestClassifier(n_estimators=ntrees, max_depth=3).fit(X, y)

In [5]:
clf.score(X, y)

0.968365553602812

# Discretizing the input data using the decision box boundaries

In [6]:
all_thresh = get_ens_thresh(clf)
data_ = disc_data(X, all_thresh)
print(data_.shape)
data_.head()

(569, 304)


,28_0.42970000207424164,26_0.31780000030994415,20_14.664999961853027,20_14.869999885559082,7_0.04988499917089939,26_0.2150000035762787,6_0.08241000026464462,2_103.29999923706055,7_0.050005000084638596,7_0.049230000004172325,...,0_15.275000095367432,1_16.109999656677246,0_15.285000324249268,7_0.0506649985909462,22_115.45000076293945,10_0.5662499964237213,27_0.13655000180006027,27_0.13234999775886536,22_106.0999984741211,1_22.454999923706055
0,True,True,True,True,True,True,True,True,True,True,...,True,False,True,True,True,True,True,True,True,False
1,False,False,True,True,True,True,True,True,True,True,...,True,True,True,True,True,False,True,True,True,False
2,False,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,False
3,True,True,True,True,True,True,True,False,True,True,...,False,True,False,True,False,False,True,True,False,False
4,False,True,True,True,True,True,True,True,True,True,...,True,False,True,True,True,True,True,True,True,False


## Variables used to define the decision boxes and the inputs

In [7]:
var_x = create_var_x(all_thresh)

# Soft Solution
Using MAX-SAT to find the solution

In [8]:
epsilon = 0.4

index = 105
sample = X[index:index+1, :]
clf.predict_proba(sample)

array([[0.81273333, 0.18726667]])

In [9]:
s, c_weights = soft_attack(clf, sample, epsilon, var_x)

In [10]:
adv_weights = get_output(s, c_weights)
np.mean([v for k,v in adv_weights.items()])

0.8459074810950605

In [11]:
clf.predict_proba(sample)[0][1]

0.18726666681551657

- x_adv is the SAT results
- x_adv_sample is the x_adv transformed to the original input space
- compare is a pandas dataframe that compares the original input and the x_adv_sample

In [12]:
x_adv, x_adv_sample, compare = get_x_adv(s, var_x, sample)
compare.sort_values(by=["diff(%)"], ascending=False)

,0,1,diff(%),bound
7,0.096010,0.059379,0.381536,"[0.0580850001424551, 0.059380000457167625]"
16,0.054730,0.075345,0.376674,"[0.07528499886393547, 0.13565000146627426]"
5,0.176500,0.112932,0.360159,"[0.09484000131487846, 0.11295000091195107]"
15,0.029120,0.039533,0.357596,"[0.039499999955296516, 0.07271000184118748]"
1,15.560000,21.056400,0.353239,"[21.05500030517578, 22.454999923706055]"
8,0.192500,0.126274,0.344033,"[-inf, 0.12639999762177467]"
3,530.200000,349.600047,0.340626,"[-inf, 349.9499969482422]"
23,827.200000,547.768494,0.337804,"[416.40000915527344, 547.8999938964844]"
27,0.198600,0.132327,0.333700,"[0.1096000000834465, 0.13234999775886536]"
6,0.207100,0.139177,0.327970,"[0.11654999852180481, 0.13920000195503235]"


Comparing the prediction of the model with the one calculated using the SAT

In [13]:
clf.predict_proba(x_adv_sample)[0][1]

0.845907481095061

# Hard Solution
Implementing the ensembling logic using SAT (summation of the vlaues of the trees)

In [14]:
epsilon = 0.3

nbits = 8

index = 105
sample = X[index:index+1, :]

In [15]:
s, c_weights, seq_num = hard_attack(clf, sample, epsilon, var_x, nbits)

In [16]:
clf.predict_proba(sample)[0][1]

0.18726666681551657

Calculating the ouput of the model using the ensembling logic

In [17]:
get_value(s.model(), seq_num, nbits , ntrees)

0.5046274509803922

In [18]:
adv_weights = get_output(s, c_weights)
np.mean([v for k,v in adv_weights.items()])

0.5061740514417657

- x_adv is the SAT results
- x_adv_sample is the x_adv transformed to the original input space
- compare is a pandas dataframe that compares the original input and the x_adv_sample

In [19]:
x_adv, x_adv_sample, compare = get_x_adv(s, var_x, sample)
compare.sort_values(by=["diff(%)"], ascending=False)

,0,1,diff(%),bound
4,0.139800,0.097854,0.300042,"[0.08199500292539597, 0.09786999970674515]"
27,0.198600,0.162646,0.181036,"[0.15890000015497208, 0.1626499965786934]"
28,0.314700,0.264935,0.158135,"[-inf, 0.26520000398159027]"
9,0.076920,0.065497,0.148500,"[0.05287500098347664, 0.06551000103354454]"
0,13.110000,15.025250,0.146091,"[15.025000095367432, 15.275000095367432]"
23,827.200000,726.920806,0.121227,"[547.8999938964844, 727.1000061035156]"
25,0.409900,0.367248,0.104055,"[0.3650500029325485, 0.367249995470047]"
1,15.560000,14.080905,0.095058,"[-inf, 14.095000267028809]"
13,34.660000,31.476535,0.091848,"[28.015000343322754, 31.479999542236328]"
21,22.400000,20.429550,0.087967,"[-inf, 20.449999809265137]"


In [20]:
clf.predict_proba(x_adv_sample)[0][1]

0.5061740514417657